# 🧪 Prédiction de fertilisants – Kaggle Challenge
**Auteur** : Arnaud Chéridi
📅 Date : Juin 2025
📍 Compétition : [Playground Series - Season 3, Episode 16](https://www.kaggle.com/competitions/playground-series-s3e16)
🎯 Objectif : Prédire les 3 fertilisants les plus probables à partir de données agronomiques synthétiques.

---

# Génération de la soumission Kaggle

Ce notebook permet de charger le meilleur modèle entraîné précédemment, de générer les prédictions sur les données de test et d’exporter un fichier au format `.csv` pour soumission sur Kaggle.

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

with open('results_summary.pkl', 'rb') as file:
    results_summary = pickle.load(file)

with open('label_y.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

In [3]:
train['Fertilizer Encoded'] = label_encoder.transform(train['Fertilizer Name'])

X = train.drop(columns=['Fertilizer Name', 'Fertilizer Encoded', 'id'])
X_val = test.drop(columns=['id'])
y = train['Fertilizer Encoded']

## Chargement des quatre meilleurs modèles

- Entrainement sur les données complètes
- Prédiction
- Sauvegarde des soumissions

In [4]:
top_4 = sorted(results_summary.items(), key=lambda x: x[1]["accuracy"], reverse=True)[:4]

for model_name, model_info in top_4:
    print(f"\n🔄 Traitement du modèle : {model_name}")

    # Récupération du modèle
    model = model_info["model"]

    # Réentraînement complet sur X, y (optionnel si déjà fait)
    model.fit(X, y)

    # Prédictions de probabilité sur les données de validation/test
    probas = model.predict_proba(X_val)

    # Récupération des indices des 3 classes les plus probables
    top3 = np.argsort(probas, axis=1)[:, -3:][:, ::-1]

    # Transformation inverse pour retrouver les noms de classes
    top3_labels = label_encoder.inverse_transform(top3.ravel()).reshape(top3.shape)

    # Construction du DataFrame de soumission
    submission = pd.DataFrame({
        "id": test["id"],  # Vérifie que test["id"] est bien aligné avec X_val
        "Fertilizer Name": [" ".join(row) for row in top3_labels]
    })

    # Nom de fichier explicite
    filename = f"submission_{model_name.replace(' ', '_').lower()}.csv"
    submission.to_csv(filename, index=False)
    print(f"Fichier enregistré : {filename}")


🔄 Traitement du modèle : GradientBoosting
Fichier enregistré : submission_gradientboosting.csv

🔄 Traitement du modèle : XGBoost


/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:58:39] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Fichier enregistré : submission_xgboost.csv

🔄 Traitement du modèle : LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002025 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 750000, number of used features: 8
[LightGBM] [Info] Start training from score -1.884866
[LightGBM] [Info] Start training from score -1.880057
[LightGBM] [Info] Start training from score -1.897538
[LightGBM] [Info] Start training from score -1.911544
[LightGBM] [Info] Start training from score -1.909121
[LightGBM] [Info] Start training from score -2.067671
[LightGBM] [Info] Start training from score -2.094845


/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Fichier enregistré : submission_lightgbm.csv

🔄 Traitement du modèle : HistGradientBoosting
Fichier enregistré : submission_histgradientboosting.csv


## Résultats Kaggle

Les prédictions des 4 meilleurs modèles ont été soumises à la plateforme Kaggle pour évaluation. Voici les scores obtenus sur l’ensemble de test :

| Modèle                     | Score public |
|---------------------------|--------------|
| Gradient Boosting         | **0.32935**  |
| XGBoost                   | 0.32785      |
| LightGBM                  | 0.32711      |
| HistGradientBoosting      | 0.32701      |

> **Gradient Boosting** obtient le meilleur score parmi les modèles testés, avec un score public de **0.32935**.

Ces performances sont relativement proches les unes des autres, ce qui suggère que les modèles se heurtent à une limite commune — probablement liée à la qualité des features. Un enrichissement des données ou un feature engineering métier plus poussé serait nécessaire pour améliorer significativement les résultats.